In [1]:
from PIL import Image

def rotate_and_scale(filename):
  src =  Image.open(f'../source/images/{filename}.png')
  for i in range(360):
    angle = i
    dst = src.rotate(angle, resample=Image.NEAREST, expand=True)
    dst = dst.crop(dst.getbbox())
    dst.save(f'../source/images/{filename}_{i}.png',bits=1,optimize=False)

In [2]:
for filename in ["pine_snow_0","arrow"]:
  rotate_and_scale(filename)